In [8]:
%config Completer.use_jedi = False
import numpy as np
import pandas as pd
import os
import multiprocessing

cores_available = multiprocessing.cpu_count()

In [9]:
import pyproj
import lightgbm as lgb

from scipy import spatial

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from fuzzywuzzy import fuzz

In [10]:
def name_fuzz_ratio_comp (df_obj, name_col, indices_col):
    name_array = df_obj[name_col].to_numpy()
    return df_obj[['name','indices']].apply(
        lambda row: [fuzz.ratio(row['name'], index_name) 
                     for index_name 
                     in name_array[row['indices']]],
        axis=1)

In [15]:
def dataset_conversion(df: pd.DataFrame, transformer: pyproj.transformer.Transformer):
    df['name'].fillna(value='',inplace=True)
    
    geocentr_cartesian = df[[ 'latitude', 'longitude']].apply(lambda x: transformer.transform( x.longitude, x.latitude, 0, radians=False), axis=1).to_numpy()
    tree = spatial.KDTree(geocentr_cartesian.tolist())
    loc_data = tree.query(geocentr_cartesian.tolist(),min(len(df),10), workers=max(cores_available-1, 1))    
    distances, indices = loc_data
    
    df [["distances", "indices"]]=pd.DataFrame(
        data = {
         "distances": distances.tolist(),
         "indices": indices.tolist()
        }, 
        columns = ["distances", "indices"]
    )
    
    df['name_comp_fuzz_score'] = name_fuzz_ratio_comp(df, 'name', 'indices')
    
    x_train_set = np.apply_along_axis(
        lambda x: list(
            zip(
                np.repeat(
                    np.arange(x[0], x[0] + 1 ), len(x[1])
                ),
                x[1],
                x[2],
                x[3]
            )
        ),
        1, 
        df[['indices', 'distances', 'name_comp_fuzz_score']].reset_index().to_numpy()
    ).reshape((-1,4)).astype(int)
    
    try:
        y_train_set = (df['point_of_interest'].to_numpy()[x_train_set[:,0]]==df['point_of_interest'].to_numpy()[x_train_set[:,1]]).astype(int)
    except:
        y_train_set = None
    
    return x_train_set, y_train_set

In [16]:
transformer = pyproj.Transformer.from_crs(
        {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},
        {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
        )

In [17]:
dataframe_train_set = pd.read_csv('/kaggle/input/foursquare-location-matching/train.csv')

In [ ]:
dataframe_test_set = pd.read_csv('/kaggle/input/foursquare-location-matching/test.csv')

In [18]:
x_train_set, y_train = dataset_conversion(dataframe_train_set, transformer)

(array([[      0,       0,       0,     100],
        [      0,  704744,     134,      23],
        [      0,  280465,     137,      32],
        ...,
        [1138811, 1023047,   32272,      30],
        [1138811,  414736,   33448,      30],
        [1138811,  471379,   34675,      31]]),
 array([1, 0, 0, ..., 0, 0, 0]))

In [19]:
x_test_set, _ = dataset_conversion(dataframe_test_set, transformer)

(array([[      0,       0,       0,     100],
        [      0,  704744,     134,      23],
        [      0,  280465,     137,      32],
        ...,
        [1138811, 1023047,   32272,      30],
        [1138811,  414736,   33448,      30],
        [1138811,  471379,   34675,      31]]),
 None)

In [ ]:
X_train, X_test = x_train_set[:,2:], x_test_set[:,2:]

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
# pd.read_csv('kaggle/input/foursquare-location-matching/pairs.csv')
# pd.read_csv('kaggle/input/foursquare-location-matching/test.csv')
# pd.read_csv('/kaggle/input/foursquare-location-matching/sample_submission.csv')
# pd.read_csv('/kaggle/input/foursquare-location-matching/train.csv')


In [ ]:
d_train = lgb.Dataset(X_train, label=y_train)

In [ ]:
params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 40
params['min_data'] = 50
params['max_depth'] = 10

In [ ]:
clf = lgb.train(params, d_train, 1000)
y_pred=clf.predict(X_test)
y_pred[y_pred >= 0.5] = 1
y_pred[y_pred < 0.5] = 0